# CSV Input Pipeline for an $\mathbb R^2$ Binary Classifier  

## Objectives
- Construst a robust input pipeline to read in the CSV files generated by *generate_data-1.ipynb*.
- Use TensorFlow's `QueueRunners` to prepare shuffled mini-batches of examples from those files.
- Flesh out the skeletal framework from *PausableTraining.ipynb* with a pauseable logistic regression classifier.

## The Input Pipeline

Note: Don't run this notebook until you've run *generate_data-1.ipynb* at least once first.

In [24]:
import os
import tensorflow as tf
notebook_dir = "cluster2D"
g = tf.Graph()

### Mini-batch Design Matrix

Get the list of .csv files in the `notebook_dir`.

In [3]:
csv_file_list = [os.path.join(notebook_dir, x) for x in os.listdir(notebook_dir) if x.endswith('.csv')]
csv_file_list

['PausableLogisticRegression/1.csv',
 'PausableLogisticRegression/3.csv',
 'PausableLogisticRegression/2.csv',
 'PausableLogisticRegression/0.csv']

Add operations to the graph which prepare mini-batches.

In [4]:
with g.as_default():
    # Create a queue of the filenames
    filename_queue = tf.train.string_input_producer(csv_file_list)
    
    # (if there are lines of metadata at the top of the file, we can skip them using this parameter)
    reader = tf.TextLineReader(skip_header_lines=0)
    
    # Create graph operations to read a line from a CSV file (key is optional, but yields good info)
    key, line_from_file = reader.read(filename_queue)
    
    # Specify the default field values (in case they're missing) and their type
    record_defaults = [[0.5], [0.5], [1.0], [tf.to_int32(0)]]
    
    # Parse the CSV string into an example tensor (x,y,w,label)
    example = tf.decode_csv(line_from_file, record_defaults=record_defaults)
    
    # Split the example into a feature vector and a label
    features = tf.stack(example[:3])
    label = example[-1]
    
    # Graph operations that produce a mini-batch (key batch is optional)
    key_batch, data_batch, label_batch = tf.train.shuffle_batch(
        [key, features, label], 
        batch_size=10, 
        capacity=400, 
        min_after_dequeue=100)

There is a lot going on in this block and the documentation for these functions isn't totally clear, but if you play with the parameters of `tf.train.shuffle_batch` and watch that `key` variable, it quickly becomes apparent what's going on:
- Producer threads continually read examples from the files and dump them into a bin (a random queue) unless the bin contains `capacity` examples (in which case they wait until it doesn't).
- Consumer threads remove randomly selected `batch_sized` examples from the bin unless it contains < `batch_size + min_after_dequeue` examples.
- You can make mini-batches indefinitely (the data are re-read).
- Each time `tf.train.shuffle_batch` is called a file is randomly chosen and examples are read from it **in sequence**, starting from the first.
- If the same file is opened multiple times, the reader doesn't pick up where it left off: it reads from the beginning again!  (So, make sure `batch_size + min_after_dequeue` $\geq$ #examples/file)

Particularly useful tests:
- If you set `min_after_dequeue=0` then each mini-batch contains the first `batch_size` examples in order (because we gave `shuffle_batch` permission to empty the queue without waiting for more than one example).  This means we have to set `min_after_dequeue` $\geq$ number of examples in each file (otherwise the examples at the end of each file will never appear in a mini-batch).
- If you set `min_after_dequeue=1` then two examples are tossed in the bin and one of them is drawn at random and replaced by the next example in the file.  Then we choose another one at random.

**Question:** Could I use `reader.read_up_to()` instead of `reader.read()` here?


Let's have a quick look at some of these mini-batches.

In [5]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)
    
    for i in range(2000):
        k, b, l = sess.run([key_batch, data_batch, label_batch])
        if i % 500 == 0:
            print(k, b, l)

    coord.request_stop()
    coord.join(threads)

[b'PausableLogisticRegression/1.csv:6'
 b'PausableLogisticRegression/1.csv:1'
 b'PausableLogisticRegression/1.csv:5'
 b'PausableLogisticRegression/2.csv:2'
 b'PausableLogisticRegression/1.csv:24'
 b'PausableLogisticRegression/3.csv:2'
 b'PausableLogisticRegression/2.csv:22'
 b'PausableLogisticRegression/3.csv:8'
 b'PausableLogisticRegression/3.csv:11'
 b'PausableLogisticRegression/2.csv:13'] [[-0.31064299  0.490621    1.        ]
 [-0.57483     0.42409301  1.        ]
 [-0.116197   -0.40730399  1.        ]
 [ 0.29039899  1.14159     1.        ]
 [-0.597516    0.380602    1.        ]
 [ 1.15666997  1.26133001  1.        ]
 [ 0.91725898  0.87689102  1.        ]
 [ 0.63346601  0.95928502  1.        ]
 [ 0.95553797  0.98903     1.        ]
 [ 0.763758    0.49739     1.        ]] [0 0 0 1 0 1 1 1 1 1]
[b'PausableLogisticRegression/3.csv:23'
 b'PausableLogisticRegression/0.csv:10'
 b'PausableLogisticRegression/2.csv:8'
 b'PausableLogisticRegression/0.csv:5'
 b'PausableLogisticRegression/1.cs

## Logistic Regression

In logistic regression over a plane, our trainable model is a vector $\boldsymbol \theta \in \mathbb R^3$ of parameters that uniquely specifies a separating hyperplane.  Our input is a set of points in $\mathbf p \in \mathbb R^2$ that we tabulate in an augmented design matrix $X \in \mathbb R^{N\times 3}$ where each row is of the form $(x_1,x_2,1)$.  The last dimension is held at a constant value of 1 in order to support a handy trick: translate points using matrix multiplication (instead of vector addition).

Our (parametrized) hypothesis function $h_{\boldsymbol \theta} (\mathbf p)$ represents the certainty with which the classifier believes $\mathbf p$ to be red instead of blue.  For example, if $h_{\boldsymbol \theta} (\mathbf q)=0.25$, that means the classifier is 75% sure that $\mathbf q$ is blue).  Hence, we can write
$$
\begin{align}
P(y = 1 \mid \mathbf p; \boldsymbol \theta) &= h_{\boldsymbol \theta} (\mathbf p)\\
P(y = 0 \mid \mathbf p; \boldsymbol \theta) &= 1-h_{\boldsymbol \theta} (\mathbf p)
\end{align}
$$

These can be condensed into a single expression,
$$
\begin{align}
P(y \mid \mathbf p; \boldsymbol \theta) &= \left(h_{\boldsymbol \theta} (\mathbf p)\right)^y \left(1-h_{\boldsymbol \theta} (\mathbf p)\right)^{(1-y)}
\end{align}
$$

The core of the hypothesis function is just the the linear functional $\varphi_\theta : \mathbb R^3 \rightarrow \mathbb R$, given by $$\varphi_\theta  = \left| \, \boldsymbol \theta \right\rangle$$

If $\varphi_\theta(\mathbf p) = \mathbf p \cdot \boldsymbol \theta$ is negative, the classifer guesses $\mathbf p$ is blue; if it's positive the classifier guesses $\mathbf p$ is red.  If it's 0, the classifier doesn't know what to think! 

The range of $\varphi_\theta$ is all of $\mathbb R$, however, so it doesn't make sense to treat it as a probability directly.  In order to do that, we need to temper it with a function that will compress all of $\mathbb R$ down into the unit interval (where probabilities live).  There are all sorts of functions that could do this, but since we're going to use gradient descent we need one which is monotonic, differentiable, and whose derivative never vanishes (because derivatives of 0 thwart gradient descent--this is called "saturation").  Furthermore, it would be wonderful if it had a derivative that was super-easy to calculate.

The logistic sigmoid fits the bill perfectly. Here it is with its derivative: $$s(t)=\frac 1 {1+e^{-t}},\ \ \ \ \ \ \frac d {dt} s(t) = s(t)(1-s(t))$$

Isn't that derivative amazing and beautiful?  Thus, our hypothesis function is
$$
h_{\boldsymbol \theta} (\mathbf p) = s\left(\varphi_\theta(\mathbf p)\right) = \frac 1 {1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}} = \frac 1 {1+e^{-\theta_0 -\theta_1 x_1 - \theta_2 x_2}}
$$

Given a ground truth label $y \in \{0,1\}^N$ (0 for "blue", 1 for "red") for point $\mathbf p$, the cross-entropy between $y$ and our estimator $h_{\boldsymbol \theta} (\mathbf p)$ is given by:
$$
\begin{align*}
H(y,h_{\boldsymbol \theta} (\mathbf p)) &= -\sum_{i=0}^1 P(y=i \mid y) \log P(y=i \mid \mathbf p; \boldsymbol \theta)\\
&= -(1-y) \log P(y=0 \mid \mathbf p; \boldsymbol \theta) - y \log P(y=1 \mid \mathbf p; \boldsymbol \theta)\\
&= -(1-y) \log (1-h_{\boldsymbol \theta} (\mathbf p)) - y \log h_{\boldsymbol \theta} (\mathbf p)
\end{align*}
$$

This is for batch gradient descent, however.  For stochastic gradient descent, we're looking at only a single training example $(\mathbf p, y) \in \mathbb R^3 \times \mathbb 2$.  The cross-entropy loss in this case is 
$$
H(y,h_{\boldsymbol \theta} (\mathbf p)) = -y \log h_{\boldsymbol \theta} (\mathbf p)
$$

For any type of gradient descent we need the gradient of the loss:
$$ 
\begin{align*}
\nabla_\theta H(y,h_{\boldsymbol \theta} (\mathbf p)) = -y \log h_{\boldsymbol \theta} (\mathbf p)
\end{align*}
$$
$$\vdots$$

Actually, you know what?  This whole derivation has blown, like, seriously over-quota.  Let's cut to the chase already and relegate all this thoroughly edge-of-your-seat calculus to the appendix.  Let me just say this (this is the really important implementation issue):

During training, we need to compute the gradient of the loss.  Now, the loss is the cross-entropy between the sigmoid/softmax of an inner product (i.e. the classifier's prediction) and the "real," ground truth labels.  So, to calculate that, first we have to compute the the classifier's prediction, then the cross-entropy, and then take the gradient, right?

BZZZZZ!  Wrong!

That's doing a whole lot of inefficient busy-work that, as an added bonus, is quite likely to give you underflows, overflows, and truncation errors (e.g. think about dividing a float by, say, $10^{-16}$ and then multiplying it by $10^{-16}$).  The secret is that the gradient of a sigmoid is really simple and a cross entropy of a sigmoid involves some logarithms of exponential functions that can be gracefully cancelled-out as long as you skip the intermediate steps and calculate the gradient directly.  

All we care about during training is the gradient anyway, so we can get away with doing this.  TensorFlow is built to encourage it!

### Trainable Parameters: $\boldsymbol \theta = (\theta_0, \theta_1, \theta_2) \in \mathbb R^3$

In [6]:
with g.as_default():
    theta = tf.get_variable(name="theta", 
                            shape=[3], 
                            initializer=tf.random_normal_initializer(mean=0, stddev=1, seed=None, dtype=tf.float32),
                            dtype=tf.float32)
    print(theta)

<tf.Variable 'theta:0' shape=(3,) dtype=float32_ref>


Let's do a quick check to make sure theta looks right when it's initialized.  If you get all zeroes, check the console for error messages.  Sometimes it fails to allocate GPU memory for whatever reason. 

In [7]:
with tf.Session(graph=g) as s:
    tf.global_variables_initializer().run()
    print(s.run([theta]))

[array([-0.64781421,  0.41394639, -1.041875  ], dtype=float32)]


### Training Operation

As mentioned earlier, despite the fact that 
$$
H(y,h_{\boldsymbol \theta} (\mathbf p)) = (y-1) \log (1-h_{\boldsymbol \theta} (\mathbf p)) - y \log h_{\boldsymbol \theta} (\mathbf p)
$$
we're better off mashing everything together and computing the gradient of the loss directly.

In [19]:
'''Hypothesis function
    Args:
        X: design matrix (N x 3)
        theta: trainable parameters (3 floats)
'''    
learning_rate = 0.1
with g.as_default():
    # Step counter
    global_step = tf.Variable(0, name="global_step", trainable=False)

    # Compute the hidden layer output (pre-sigmoid function)
    logits = tf.reshape(tf.matmul(data_batch, theta[:,None]), [-1])
    
    # Compute the loss of each example in the mini-batch
    losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(label_batch, tf.float32), logits=logits)
    
    # Find the mean loss of the mini-batch
    loss = tf.reduce_mean(losses)
    
    # Train Op
    train_op = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(loss, global_step=global_step)

(10,)


#### Test train_op in a simple session

Just for a sanity check, let's run our train_op in an old-fashioned tf.Session.  For the real training, we'll use a tf.train.Supervisor.managed_session() 

In [23]:
with tf.Session(graph=g) as s:
    s.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=s)
    
    for i in range(500):
        t_op, ls, gstep = s.run([train_op, loss, global_step])
        if (i % 100 == 0):
            print("step %3d:\tloss = %f" % (gstep, ls))

    coord.request_stop()
    coord.join(threads)

step   1:	loss = 1.407199
step 101:	loss = 0.340095
step 201:	loss = 0.133344
step 301:	loss = 0.361933
step 401:	loss = 0.375085


Looks good.  Now let's do it with a pausable session.

### Training Session

TensorFlow supports saving and loading the network parameters using the `tf.train.Saver` class. Since we want checkpoints, however, we can make this easier by using a `tf.train.Supervisor.managed_session` instead of the usual `tf.Session`.  

A `managed_session` has its own Saver and it will save checkpoints automatically and reload from them.  The only thing it leaves for us to do is to implement is the "pause button."  To do that, we'll periodically check the training directory for a file named "pause".  If one exists, we'll delete it (in preparation for the next use), save a checkpoint manually, and then shut down the training.

**BEFORE RUNNING THE NEXT CELL** get a terminal open and have this command ready to go:
```
touch /tmp/pausable_training/pause
```

**NB**: Only the *values* of the network parameters are saved and reloaded--not the network topology itself.  We still have to build the graph and if there's input (which there usually is), load it.  The step we skip is the network initialization step (`tf.global_variables_initializer().run()`).  Obviously, we have to leave that to `managed_session()` since we want it to handle the task of choosing whether to reload values from storage or initialize them from scratch.

In [ ]:
import time
training_dir = "/tmp/pausable_training/"
pause_file = training_dir + "pause"
checkpoint_file = training_dir + "pt.ckpt"

with graph.as_default():
    sv = tf.train.Supervisor(logdir=training_dir)
    with sv.managed_session() as s:

        # Supervisor calls tf.global_variables_initializer().run() for us

        while not sv.should_stop():
            _, y, gstep = s.run([train_op, p, global_step])
            if (gstep % 100 == 0):
                print("step %3d:\tp = %f" % (gstep,y))
                if (tf.gfile.Exists(pause_file)):
                    sv.Stop()
                time.sleep(1)
                
        if tf.gfile.Exists(pause_file):
            print("Pause command received.  Saving checkpoint and shutting down.")
            tf.gfile.Remove(pause_file)
            sv.saver.save(s, checkpoint_file, global_step=global_step)

INFO:tensorflow:Restoring parameters from /tmp/pausable_training/pt.ckpt-1900
INFO:tensorflow:global_step/sec: 0
step 2000:	p = 4000.000000
step 2100:	p = 4200.000000
step 2200:	p = 4400.000000
step 2300:	p = 4600.000000
step 2400:	p = 4800.000000
step 2500:	p = 5000.000000
Pause command received.  Saving checkpoint and shutting down.


### Now, we shut down and continue with more training later

In [ ]:
# Kill the kernel, forcing it to restart - NB: You'll have to step manually from here on.
import os
os._exit(0)

Re-run the notebook at this point to perform subsequent training sessions.  Each time it will pick up where it left off (unless the training directory has been wiped).  Note the reported "step" numbers above to see it resuming.

### Next Objectives
#### Make this a little less skeletal 
Use input data and actually train some simple parameters (call `minimize`) so we have a real train_op.  Try logistic regression on a randomly-generated dataset.
#### Get a better handle on TensorBoard reporting  
Want to see the loss and the trainable parameters at the very least.  Eventually we'll add learning rates, kernels, etc.
#### Keep a roster of the top performers
Keep snapshots of the networks corresponding to local minima in the training loss.  Make it easy to load those snapshots.  Once we suspect we're beginning to overtrain the network, we can load up those "training highlights", cross-validate them, and package up the best one for "release" (e.g. Kaggle submission).
#### Use a file input queue
We want to operate on lots of data: more then what will fit main memory.  See this page: https://www.tensorflow.org/programmers_guide/reading_data

# Appendix (under construction)

The gradient of the loss is, therefore,
$$\begin{eqnarray}
\nabla_{\boldsymbol \theta} \ell(\mathbf p) &=& \left(\frac \partial {\partial \theta_0} \ell(\mathbf p), \frac \partial {\partial \theta_1} \ell(\mathbf p), \frac \partial {\partial \theta_2} \ell(\mathbf p)\right)\\
\end{eqnarray}$$

$$
\begin{align*}
H(y,h_{\boldsymbol \theta} (\mathbf p)) &= -\sum_{i=0}^1 P(y=i \mid y) \log P(y=i \mid \mathbf p; \boldsymbol \theta)\\
&= -(1-y) \log P(y=0 \mid \mathbf p; \boldsymbol \theta) - y \log P(y=1 \mid \mathbf p; \boldsymbol \theta)\\
&= -(1-y) \log \left(1-h_{\boldsymbol \theta} (\mathbf p)\right) - y \log h_{\boldsymbol \theta} (\mathbf p)\\
&= -(1-y) \log \left(1-\frac 1 {1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}} \right) - y \log \frac 1 {1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}}\\
&= -(1-y) \log \left(\frac {e^{-\langle \boldsymbol \theta, \mathbf p\rangle}} {1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}} \right) - y \log \frac 1 {1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}}\\
&= (y-1) \left[ \log e^{-\langle \boldsymbol \theta, \mathbf p\rangle} - \log \left(1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right) \right] + y \left[ \log \left(1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right) \right]\\
&= y \log e^{-\langle \boldsymbol \theta, \mathbf p\rangle} - y \log \left(1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right) - \log e^{-\langle \boldsymbol \theta, \mathbf p\rangle} + \log \left(1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right) + y \log \left(1+e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right)\\
&= (y-1) \log e^{-\langle \boldsymbol \theta, \mathbf p\rangle} + \log \left(1 + e^{-\langle \boldsymbol \theta, \mathbf p\rangle}\right)
\end{align*}
$$
